In [1]:
import phil_univax
import ast, itertools
import numpy as np
import concurrent.futures

In [2]:
opt_params_template = '''
{
'vaccine_total_avail[%(vaccine_id)d]' : '%(total_doses)d',
'vaccine_additional_per_day[%(vaccine_id)d]' : '%(daily_doses)d',                                                                                                      
'vaccine_dose_efficacy_values[%(vaccine_id)d][0]' : '1 %(efficacy)f',
'vaccine_coverage_age_map_values[%(vaccine_id)d]' : '5 %(coverage)s'
}
'''

In [3]:
def parameterize(years, coverage, universal_efficacy, seasonal_efficacy, proportion_universal):
    assert(years<=8)
    n = 1188112
    n_19_50 = 483985
    p = dict(days=360*years)
    
    base_coverage = [0.0795, 0.2806, 0.2204, 0.1907, 0.2289]   
    
    universal_doses_19_50 = int(n * coverage * base_coverage[2] * proportion_universal)
    
    p.update(ast.literal_eval(opt_params_template % dict(
        vaccine_id=0,
        total_doses=universal_doses_19_50,
        daily_doses=universal_doses_19_50,
        efficacy=universal_efficacy,
        coverage='0.0001 0.0001 0.9996 0.0001 0.0001'
    )))
    
    total_doses = n*coverage
    baseline_seasonal_doses = [x*total_doses for x in base_coverage]
    adjusted_seasonal_doses = baseline_seasonal_doses[:]
    adjusted_seasonal_doses[2] = adjusted_seasonal_doses[2] - universal_doses_19_50
    seasonal_doses = int(sum(adjusted_seasonal_doses))
    seasonal_coverage = [x/seasonal_doses for x in adjusted_seasonal_doses]
    
    
    p.update(ast.literal_eval(opt_params_template % dict(
        vaccine_id=1,
        total_doses=seasonal_doses,
        daily_doses=seasonal_doses,
        efficacy=seasonal_efficacy,
        coverage=' '.join(['%f'%x for x in seasonal_coverage])
    )))
    
    for vaccine_id in range(2,7):
        p.update(ast.literal_eval(opt_params_template % dict(
            vaccine_id=vaccine_id,
            total_doses=int(n*coverage),
            daily_doses=int(n*coverage),
            efficacy=seasonal_efficacy,
            coverage=' '.join(['%f'%x for x in base_coverage])
        )))
    return p

In [4]:
#coverage_values = [0.4,0.5,0.6]
#universal_efficacy_values = [0.40]
#seasonal_efficacy_values = [0.40, 0.60]
#proportion_universal_values = [0.00,0.25,0.50,0.75,1.00]

In [6]:
coverage_values = [0.4,0.5]
universal_efficacy_values = [0.20,0.30,0.40]
seasonal_efficacy_values = [0.60]
proportion_universal_values = [0.00,0.50,1.00]

In [ ]:
reload(phil_univax)
pux = phil_univax.PhilUniversalExperiment()
    
results = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
    for i in range(4):
        for (coverage, universal_efficacy, seasonal_efficacy, proportion_universal) in itertools.product(
                 coverage_values, universal_efficacy_values,
                 seasonal_efficacy_values, proportion_universal_values):
            opt_params = parameterize(4,
                coverage, universal_efficacy, seasonal_efficacy, proportion_universal)
            run_name = '%f_%f_%f_%f' % (
                coverage, universal_efficacy, seasonal_efficacy, proportion_universal)
            results[run_name] = executor.submit(pux.run, 'params.universal', opt_params, run_name)

with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
    for k,v in results.items():
        if v.result() is False:
            (coverage, universal_efficacy,
             seasonal_efficacy, proportion_universal) = [float(x) for x in k.split('_')]
            opt_params = parameterize(4,
                coverage, universal_efficacy, seasonal_efficacy, proportion_universal)
            run_name = '%f_%f_%f_%f' % (
                coverage, universal_efficacy, seasonal_efficacy, proportion_universal)
            results[run_name] = executor.submit(pux.run, 'params.universal', opt_params, run_name)            


In [8]:
def split_name(d):
    d = d.copy()
    d1 = d.name.str.extract(r'/(\d.+)\.(\d+)', expand=True)
    d2 = d1[0].str.split('_', expand=True)
    d['uid'] = d1[1]
    cols = ('coverage', 'universal_efficacy', 'seasonal_efficacy', 'proportion_universal')
    for i,c in zip(range(4), cols):
        d[c] = d2[i].astype(float)
    del d['name']
    return d 

def summarize2(d):

    def summarize_by_age(d):
        return pd.Series({
            'individuals': d.N_p.mean(),
            'vaccines': d.V_i.sum(),
            'infections': d.IS_i.sum()
        })

    d1 = split_name(d)
    d1['year'] = (d1.day / 360).astype('int')
    
    d1 = d1.groupby(
        ['uid','year', 'age','coverage', 'universal_efficacy', 'seasonal_efficacy', 'proportion_universal']
    ).apply(summarize_by_age).reset_index()
    
#    d1 = d1.groupby(['year', 'age','coverage', 'universal_efficacy',
#           'seasonal_efficacy', 'proportion_universal'], as_index=False).mean()
    #d1['age_specific_coverage'] = d1['vaccines'] / d1['individuals']
    d1['attack_rate'] = d1['infections'] / d1['individuals']
    return d1

In [9]:
import pandas as pd
summarize2(pd.read_csv(
    'poe_output.csv')).to_csv(
    'initial-results.csv', index=False)

In [10]:
!head initial-results.csv

uid,year,age,coverage,universal_efficacy,seasonal_efficacy,proportion_universal,individuals,infections,vaccines,attack_rate
113735888290461303,0,"[0, 5)",0.5,0.3,0.6,0.5,63016.0,11674.0,43891.0,0.18525453853
113735888290461303,0,"[18, 50)",0.5,0.3,0.6,0.5,483985.0,22929.0,137664.0,0.0473754351891
113735888290461303,0,"[5, 18)",0.5,0.3,0.6,0.5,185902.0,21056.0,174195.0,0.113263977795
113735888290461303,0,"[50, 65)",0.5,0.3,0.6,0.5,255198.0,14764.0,114204.0,0.0578531179711
113735888290461303,0,"[65, 106)",0.5,0.3,0.6,0.5,200011.0,15209.0,143607.0,0.076040817755
113735888290461303,1,"[0, 5)",0.5,0.3,0.6,0.5,63016.0,6349.0,39427.0,0.10075218992
113735888290461303,1,"[18, 50)",0.5,0.3,0.6,0.5,483985.0,11888.0,137017.0,0.0245627447132
113735888290461303,1,"[5, 18)",0.5,0.3,0.6,0.5,185902.0,8170.0,162975.0,0.0439478865209
113735888290461303,1,"[50, 65)",0.5,0.3,0.6,0.5,255198.0,8251.0,113347.0,0.0323317580859
